# Smart Car Buyer AI Agent

## Overview

This notebook presents an intelligent car-buying assistant using LangGraph and an LLM model. The system assists users in defining their car-buying needs, refines search filters, and retrieves relevant listings, providing a streamlined buying experience.

In [ ]:
%pip install langgraph
%pip install langchain
%pip install langchain-openai
%pip install importnb
%pip install python-dotenv
%pip install patchright
%pip install lxml
%pip install playwright

In [ ]:
# Import necessary libraries
from typing import TypedDict, Dict, List, Any
from langgraph.graph import StateGraph, END
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from IPython.display import display, Image
from langchain_core.runnables.graph import MermaidDrawMethod
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage

from langsmith import Client
from langsmith import traceable
from langsmith.wrappers import wrap_openai
import openai
import asyncio
from importnb import Notebook

import os
from dotenv import load_dotenv

# This import is required only for jupyter notebooks, since they have their own eventloop
import nest_asyncio
nest_asyncio.apply()

with Notebook():
    from scrapers.autotrader import AutotraderInterface, WebsiteInterface


In [ ]:
!playwright install
!patchright install chromium

In [ ]:
# Load environment variables
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')
os.environ["LANGCHAIN_TRACING_V2"] = "false"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"] = "car_buyer_agent"
os.environ["LANGCHAIN_API_KEY"] = os.getenv('LANGCHAIN_API_KEY', "")

# Initialize the language model
llm = ChatOpenAI(model="gpt-4o-mini")

In [ ]:
# Initialize the LangSmith client
langsmith_client = Client()

openai_client = wrap_openai(openai.Client())

In [ ]:
class State(TypedDict):
    """Represents the state of the car-buying process."""
    user_needs: str
    web_interfaces: List[WebsiteInterface]
    listings: List[Dict[str, str]]
    selected_listing: Dict[str, str]
    additional_info: Dict[str, str]
    user_input: str
    next_node: str
    messages: List

In [ ]:
def ask_user_needs(state: State) -> State:
    """Ask user initial questions to define their needs for the car."""
    
    messages = state.get("messages", [])
    
    system_message = ""
    
    if len(messages) == 0:
        system_message += "You are a car buying assistant. Your goal is to help the user find a car that meets their needs. Start by introducing yourself and asking about their requirements, such as intended usage (e.g., commuting, family trips), budget, size preferences, and any specific constraints or features they value. Use their responses to guide them toward the best options."
    else:
        system_message += "Ask the user for any additional information that can help narrow down the search. If he asked any questions before, answer them before asking for more information."
        
        
    # Check if we already have some user needs to preserve
    existing_needs = state.get("user_needs", "")
    if existing_needs:
        system_message += f" Here's what we know about the needs of the user so far:\n\n{existing_needs}"
    
    messages.append(SystemMessage(system_message))
    
    # Get message from the LLM
    response = llm.invoke(messages).content
    
    print(response)
    
    messages += [AIMessage(response)]
    
    messages += [HumanMessage(input(response))]
    
    messages += [
        SystemMessage(
            "Summarize the user's car-buying needs in one clear and concise sentence based on their input and any prior knowledge.\n"
            "Provide the summary as the first line of your response.\n"
            "Then, skip two lines and indicate the next step by writing either 'ask_user_needs' or 'build_filters':\n"
            "- Use 'ask_user_needs' if you need more information or if the user asked a question.\n"
            "- Use 'build_filters' if you have enough details to search for cars online.\n"
            "Do not write anything else. Reserve questions and explanations for later."
        )
    ]
    
    response = llm.invoke(messages).content
    
    messages += [AIMessage(response)]

    state["user_needs"] = response.split("\n")[0].strip()
    
    print("\nI have summarized your car-buying needs as follows:")
    print(state["user_needs"])
    
    if len(response.split("\n")) > 1:
        state["next_node"] = response.split("\n")[-1].strip()
    else:
        state["next_node"] = "build_filters"
        
    print(f"\nNext node: {state['next_node']}")

    return state

In [ ]:
def build_filters(state: State) -> State:
    """Build and refine search filters based on user needs."""
    
    for interface in state["web_interfaces"]:
        filters_info = interface.get_filters_info()
        
        # TODO: Check if this website is useful to the user based on the filters
        # If not continue to the next interface
        
        # If the website is useful, use LLM to setup the filters based on user needs
        
        # Define system instructions with filters information
        system_message = SystemMessage(filters_info + "\n\n" + "User needs:\n" + state["user_needs"])

        # Use the LLM to process the user's needs and set the filters
        try:
            result = llm.invoke([system_message])
            llm_response = result.content.strip()

            # Validate and set the filters for the interface
            interface.set_filters_from_llm_response(llm_response)
            print(f"\nSuccessfully set filters for: {interface.base_url}")
            print(f"Updated URL: {interface.url}")
        except ValueError as e:
            print(f"Failed to set filters for {interface.base_url}: {e}")
        except Exception as e:
            print(f"An error occurred while processing filters for {interface.base_url}: {e}")
    
    return state

In [ ]:
async def fetch_listings_from_sources(web_interfaces: List[WebsiteInterface]) -> List[Dict[str, str]]:
    """Simulate retrieval of car listings from LaCentrale and mobile.de based on filters.
    
    Args:
        filters (dict): Dictionary containing search filters (e.g., budget, fuel type).
        
    Returns:
        list: A list of dictionaries, each representing a car listing.
    """
    listings = []
    for interface in web_interfaces:
        listings += await interface.crawl()
        
    return listings

In [ ]:
def search_listings(state: State) -> State:
    """Search for cars on LaCentrale and mobile.de based on filters."""
    """Display the first listings for the user to view."""
    """Synchronous wrapper for search_listings."""
    async def _search_listings():
        return await fetch_listings_from_sources(state["web_interfaces"])
    
    listings = asyncio.run(_search_listings())
    state["listings"] = listings
    
    print(f"Successfully fetched {len(listings)} listings from the sources.")
    
    ai_message = ""
    
    # Display the first few listings for the user to view
    ai_message += "Here are recent listings that match your requirements:\n"
    for i, listing in enumerate(state["listings"][:5], 1):
        ai_message += f"{i}.\n"
        for key, value in listing.items():
            formatted_key = key.replace("_", " ").capitalize()
            ai_message += f"   {formatted_key}: {value}\n"
        ai_message += "\n"  # Add an extra line for readability
    
    user_prompt = "Would you like to view more details about a specific listing, or refine your search?"
    ai_message += user_prompt
    
    print("\033[92m" + ai_message + "\033[0m")
        
    state["messages"].append(AIMessage(ai_message))
    
    # Ask the user for input
    state["messages"].append(HumanMessage(input(user_prompt)))
    
    state["messages"].append(SystemMessage("Based on the user's response, provide the next step by writing either 'select_listing LISTING_ID', 'refine_search' or 'end_conversation'."))
    
    response = llm.invoke(state["messages"]).content
    
    state["messages"].append(AIMessage(response))
    
    if "select" in response.lower():
        state["next_node"] = "fetch_additional_info"
        selected_listing_id = response.split()[1].strip()
        for i, listing in enumerate(state["listings"][:5], 1):
            if selected_listing_id in listing["id"]:
                state["selected_listing"] = listing
                break
    elif "refine" in response.lower():
        state["next_node"] = "ask_user_needs"
    else:
        state["next_node"] = END
        
    return state

In [ ]:
def fetch_additional_info(state: State) -> State:
    """Fetch more details about the selected car listing."""
    listing = state["selected_listing"]
    prompt = SystemMessage(
        f"Provide additional information about this car: {listing['title']}, "
        f"including engine specifications, common issues with this model, and market value."
    )
    
    result = llm.invoke([prompt])
    
    listing["additional_info"] = result.content
    
    print(f"\033[92mHere is additional information about the selected car:\n{listing['additional_info']}\n\033[0m")
    
    # TODO: Decide either te refine the search or end the conversation
    state["next_node"] = END
    
    return state

In [ ]:
# Initialize the StateGraph
workflow = StateGraph(State)

# Define the nodes in the graph
workflow.add_node("ask_user_needs", ask_user_needs)
workflow.add_node("build_filters", build_filters)
workflow.add_node("search_listings", search_listings)
workflow.add_node("fetch_additional_info", fetch_additional_info)

# Define edges
workflow.add_conditional_edges("ask_user_needs", lambda state: state["next_node"], ["build_filters", "ask_user_needs"])
workflow.add_edge("build_filters", "search_listings")
workflow.add_conditional_edges("search_listings", lambda state: state["next_node"], ["fetch_additional_info", "ask_user_needs", END])

# Set the entry and exit points
workflow.set_entry_point("ask_user_needs")
workflow.add_conditional_edges("fetch_additional_info", lambda state: state["next_node"], ["ask_user_needs", END])


# Compile the workflow
app = workflow.compile()

In [ ]:
display(
    Image(
        app.get_graph().draw_mermaid_png(
            draw_method=MermaidDrawMethod.API,
        )
    )
)

In [ ]:
# Verify initial setup and function invocation
def run_car_buyer_agent():
    """Run the car-buying assistant with LangGraph."""
        
    messages = []
    
    initial_state = State(
        user_needs={}, 
        web_interfaces=[AutotraderInterface()], 
        listings=[],
        selected_listing={}, 
        additional_info={},
        user_input="",
        next_node="",
        messages=messages
    )
    result = app.invoke(initial_state)
    return result

# Execute the agent
car_buyer_result = run_car_buyer_agent()

# Print result for debugging purposes
print("Car Buyer Result:", car_buyer_result)

In [ ]:
# Display summary of the final recommendation
if "selected_listing" in car_buyer_result:
    listing = car_buyer_result["selected_listing"]
    print(f"\nFinal Recommendation:\n{listing['title']} - {listing['price']} - {listing['mileage']} km")
    print("Additional Information:")
    for key, value in car_buyer_result["additional_info"].items():
        print(f"{key}: {value}")
else:
    print("No car listing selected.")